<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Yahoo_Data_Downloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
import json as js

today = datetime.today()
delta60 = today - timedelta(days=60)
delta120 = today - timedelta(days=120)
data_path = '/content/drive/My Drive/ml-trde-notebooks/data'
tickers = ["SPY", "TSLA", "AAPL", "IBM", "F", "CAT", "BAC", "B", "META", "AMZN", "XOM", "BP"]
coins = ["FCON-USDT", "GMT3L-USDT", "NEAR3L-USDT", "H2O-USDT", "DOGE3L-USDT", "DOGE3S-USDT"]
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"


In [54]:


def fetch_klines(symbol, interval, start_time, end_time):

    klines = []

    # Calculate the maximum number of bars that can be retrieved per request
    limit = 1500

    interval_t = 86400

    if interval == "15min":
      interval_t = 900
    elif interval == "1day":
      interval_t = 86400
    else:
      raise Exception(interval + " is unimplemented, add a mapping")

    # Keep sending requests until we have retrieved all of the data
    while start_time < end_time:
        
        print("Fetching: ", symbol, "for range: " , datetime.fromtimestamp(end_time), " to: ", datetime.fromtimestamp(start_time))
        
        # Send the request to the API
        response = requests.get(
            KUCOIN_CANDLES,
            params={
                "symbol": symbol,
                "type": interval,
                "startAt": start_time,
                "endAt": end_time,
            }
        )

      
        # Append the klines to the list
        response_text = response.text
        response_data = js.loads(response_text);

        # Check for errors
        if (response_data["code"] != "200000"):
          raise Exception("Illegal response: " + response_text)

        # Update the start and end times for the next request
        start_time = start_time - limit * interval_t
        end_time = start_time

        # Pause for a moment to avoid hitting the rate limit
        time.sleep(3)

    return response_data


In [59]:
# minute bars
for ticker in tickers:
    tickerObj = yf.download(tickers = ticker, interval = "15m", start=delta60, end=today)
    df = pd.DataFrame(tickerObj)
    with open(data_path + "/" + ticker + "-15.csv", 'w', encoding = 'utf-8-sig') as f:
      df.to_csv(f)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
# day bars
for ticker in tickers:
    tickerObj = yf.download(tickers = ticker, interval = "1d")
    df = pd.DataFrame(tickerObj)
    with open(data_path + "/" + ticker + ".csv", 'w', encoding = 'utf-8-sig') as f:
      df.to_csv(f)

  

In [58]:
#download coins
now = int(time.time())
start_time = now - (365 * 24 * 60 * 60) *3 # seconds in a year *. 3 years
end_time = now
interval = "1day"
# Fetch the klines
for coin in coins:
  klines = fetch_klines(coin, interval, start_time, end_time)
  df_history = pd.DataFrame(klines["data"])
  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history = df_history.rename(columns={"time":"Date", "open":"Open", "close":"Close", "high":"High", "low":"Low", "volume": "Volume"})
  with open(data_path + "/" + coin + ".csv", 'w', encoding = 'utf-8-sig') as f:
    df_history.to_csv(f)

Fetching:  FCON-USDT for range:  2023-01-08 23:29:04  to:  2020-01-09 23:29:04
Fetching:  GMT3L-USDT for range:  2023-01-08 23:29:04  to:  2020-01-09 23:29:04
Fetching:  NEAR3L-USDT for range:  2023-01-08 23:29:04  to:  2020-01-09 23:29:04
Fetching:  H2O-USDT for range:  2023-01-08 23:29:04  to:  2020-01-09 23:29:04
Fetching:  DOGE3L-USDT for range:  2023-01-08 23:29:04  to:  2020-01-09 23:29:04
Fetching:  DOGE3S-USDT for range:  2023-01-08 23:29:04  to:  2020-01-09 23:29:04
